# Spatially Assign Work
In this example, assignments will be assigned to specific workers based on the city district that it falls in. A layer in ArcGIS Online representing the city districts in San Diego will be used.

**Note:** This example requires having Arcpy or Shapely installed in the Python environment.

In [ ]:
from datetime import datetime
from arcgis.gis import GIS
from arcgis.geometry import Geometry
from arcgis.mapping import WebMap
from arcgis.apps import workforce

### Connect to organization and Get the Project
Let's connect to ArcGIS Online and get the Project with assignments.

In [ ]:
gis = GIS("https://arcgis.com", "workforce_scripts")
item = gis.content.get("1f7b42024da544f6b1e557889e858ac6")
project = workforce.Project(item)

### Get Layer of City Districts
Let's get the layer representing city districts and display it.

In [ ]:
districts_layer = gis.content.get("08eecb2f46c740aaba316cef8a719a88").layers[0]
districts_map = gis.map("San Diego, CA", zoomlevel=12)
districts_map.add_layer(districts_layer)
districts_map

### Add Assignments to the Map

In [ ]:
districts_map.add_layer(project.assignments_layer)

### Assign Assignments Based on Which District They Intersect
Let's fetch the districts layer and query to get all of the districts. Then, for each unassigned assignment intersect the assignment with all districts to determine which district it falls in. Assignments in the yellow district (3) should be assigned to James. Assignments in the red district (8) should be assigned to Aaron. Finally update all of the assignments using "batch_update".

In [ ]:
aaron = project.workers.get(user_id="aaron_nitro")
james = project.workers.get(user_id="james_nitro")

In [ ]:
# Fetch the districts
districts_df = districts_layer.query().df
# Fetch the unassigned assignments
assignments = project.assignments.search("status=0")
for assignment in assignments:
    # Intersect the assignment with each district
    contains_df = districts_df.contains(Geometry(assignment.geometry))
    # Determine which distict the assignment falls in
    containers = districts_df[contains_df]
    
    # If the assignment intersects any district
    if not containers.empty: 
        # Get the district number of the first intersecting district
        section = containers['district'].iloc[0]
        # Assign the assignment based on the district it's in
        if section == 3:
            assignment.worker = james
            assignment.status = "assigned"
            assignment.assigned_date = datetime.utcnow()
        elif section == 8:
            assignment.worker = aaron
            assignment.status = "assigned"
            assignment.assigned_date = datetime.utcnow()

# Update the assignments in the project
assignments = project.assignments.batch_update(assignments)

### Verify Assignments are Assigned

In [ ]:
webmap = gis.map("San Diego", zoomlevel=14)
webmap.add_layer(project.assignments_layer)
webmap